## Data Cleaning of Code Interpreter Labeled Summarizartion Superset of Wikipedia Toxic Comments Dataset

I used code interpreter to take the wikipedia toxic comments dataset and label the raw text data
with brief summaries of why the innappropriate comments of the dataset are innapropriate. There is
a text file in this directory (code_interpreter_data_prep.txt) which contains the prompts I used to perform this task. One issue of
the resultant dataset is that code interpreter was not able to fully remove the non innapropriate
comments (which was one of the things I asked it to do). So in this notebook I am going to use the 
rule adherance classifier I trained previously to filter out only the innapropriate comments for 
use in the summary model.

In [4]:
import pandas as pd

#load csv
summary_data = pd.read_csv('C:\\Users\\hunte\\OneDrive\\Documents\\Coding Projects\\Bot_Discord_Proj\\clean_summary_model\\summarization_data_prep\\code_interpreter_summarization_dataset.csv')
summary_data.head()


,text,reason
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,contains derogatory swear words
1,Hey... what is it..\n@ | talk .\nWhat is it......,contains inappropriate language or context
2,"Bye! \n\nDon't look, come or think of comming ...",contains inappropriate language or context
3,You are gay or antisemmitian? \n\nArchangel WH...,contains potentially offensive group references
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",contains derogatory swear words


In [5]:
len(summary_data)

16205

## Using RAC to Clean Data
I am going to load in the rule adherance classifier from run_local_RAC.py which contains a class that will use the RAC model trained for inference of whether strings are appropriate or innapropriate on the local machine.

In [6]:
import os 

os.chdir("C:\\Users\\hunte\\OneDrive\\Documents\\Coding Projects\\Bot_Discord_Proj\\Content-Mod-DiscordBot\\bots")
print(os.listdir())

['bot.py', 'run_local_RAC.py', 'tester', '__pycache__']


Instantiating the Rule Adherance Classifier

In [7]:
from run_local_RAC import RAC

local_RAC = RAC()

Looks to be working

In [8]:
inappropriate = "That is so dumb and stupid you suck!"

appropriate = "I don't agree with you."

inappropriate_pred, appropriate_pred = local_RAC.run(inappropriate), local_RAC.run(appropriate)

for pred in [ [inappropriate, inappropriate_pred], [appropriate, appropriate_pred] ]:
    prediction = "inappropriate" if pred[1] == 1 else "appropriate"
    print(f"For the sentence '{pred[0]}', the model predicts {prediction}.")


For the sentence 'That is so dumb and stupid you suck!', the model predicts inappropriate.
For the sentence 'I don't agree with you.', the model predicts appropriate.


Here I am mapping the local_RAC.run() method to the listed summary_data["text"] column of the df.

In [ ]:
inference_map = map(local_RAC.run, list(summary_data["text"])) 
inference_list = list(inference_map)

Below I convert the list of inferences on the the raw text data to a pandas boolean series. Then I use this series to filter out only those inappropriate comments.

In [11]:
bool_series = pd.Series(inference_list).astype(bool)
cleaned_summary_data = summary_data[bool_series].reset_index(drop=True)

In [12]:
num_removed = len(summary_data) - len(cleaned_summary_data)
print(f'{num_removed} rows removed')

2463 rows removed


In [13]:
cleaned_summary_data.head()


,text,reason
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,contains derogatory swear words
1,You are gay or antisemmitian? \n\nArchangel WH...,contains potentially offensive group references
2,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",contains derogatory swear words
3,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,contains derogatory swear words
4,Stupid peace of shit stop deleting my stuff as...,"contains derogatory swear words, contains dero..."


In [16]:
#save csv
cleaned_summary_data.to_csv('summary_dataset.csv', index=False)